In [1]:
%cd ..

c:\Users\ernes\Documents\ML Projects\ds-project-test\ds-project-test


In [2]:
%pwd

'c:\\Users\\ernes\\Documents\\ML Projects\\ds-project-test\\ds-project-test'

In [3]:

from src.ds_project.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH
from src.ds_project.utils.utils import read_yaml, create_directories
from src.ds_project import logger
from urllib.request import urlretrieve
import shutil

2025-04-28 15:50:57 - INFO: __init__ - Logging setup complete.


In [7]:
# download a file from a url using curl
!curl -O http://www3.dsi.uminho.pt/pcortez/wine/winequality.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:03 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:04 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:05 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:06 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:07 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:08 --:--:--     0curl: (6) Could not resolve host: www3.dsi.uminho.pt

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    directory_path: Path
    source_url: str
    local_data_file: Path
    raw_data_path: Path
    train_data_path: Path
    test_data_path: Path

In [ ]:
class ConfigurationManager:
    def __init__(self, 
                 config_path=CONFIG_FILE_PATH, 
                 params_path = PARAMS_FILE_PATH, 
                 schema_path = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        self.schema = read_yaml(schema_path)
        print(self.config)
        print(self.config['artifacts_dir'])
        
        # Create artifact directories if they don't exist
        if not Path(self.config['artifacts_dir']).exists():
            create_directories([self.config['artifacts_dir']])
            logger.info(f"Created artifacts directory at {self.config['artifacts_dir']}")
        else:
            logger.info(f"Artifacts directory already exists at {self.config['artifacts_dir']}")


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config= self.config['data_ingestion']
        create_directories([config['directory'], config['raw_data_path'], config['train_data_path'], config['test_data_path']], True)


        data_ingestion_config = DataIngestionConfig(
            directory_path= Path(config['directory']),
            source_url= config['source_url'],
            local_data_file= Path(config['local_data_file']),
            raw_data_path= Path(config['raw_data_path']),
            train_data_path= Path(config['train_data_path']),
            test_data_path= Path(config['test_data_path'])
        )
        return data_ingestion_config
    


In [28]:
import requests
import zipfile
import os
from src.ds_project import logger

In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            logger.info(f"Downloading data from {self.config.source_url} to {self.config.local_data_file}")
            filename, headers = urlretrieve(url=self.config.source_url, 
                                            filename=self.config.local_data_file)
            logger.info(f"Data downloaded to {self.config.local_data_file}")
        else:
            logger.info(f"Data already exists at {self.config.local_data_file}. Skipping download.")
    

    def extract_data(self):
        with zipfile.ZipFile(self.config.raw_data_file, 'r') as zip_ref:
            zip_ref.extractall(self.config.raw_data_path)
        logger.info(f"Data extracted to {self.config.raw_data_path}")

    def setup_data(self):
        # move the data to the raw data path
        if not self.config.local_data_file in os.listdir(self.config.raw_data_path):
            shutil.move(self.config.local_data_file, self.config.raw_data_path)
            logger.info(f'Data moved to {self.config.raw_data_path}')

In [67]:
manager = ConfigurationManager()
data_ingestion_config = manager.get_data_ingestion_config()
data_ingestion = DataIngestion(config=data_ingestion_config)
data_ingestion.download_data()
data_ingestion.setup_data()

2025-04-28 18:51:58 - INFO: utils - YAML file config\config.yaml loaded successfully.
2025-04-28 18:51:58 - INFO: utils - YAML file parameters.yaml loaded successfully.
2025-04-28 18:51:58 - INFO: utils - YAML file schema.yaml loaded successfully.
{'artifacts_dir': 'artifact/', 'data_ingestion': {'directory': 'artifact/data_ingestion', 'source_url': 'https://github.com/MainakRepositor/Datasets/blob/master/Titanic.csv', 'local_data_file': 'artifact/data_ingestion/Titanic.csv', 'unzip_dir': 'artifact/data_ingestion', 'raw_data_path': 'artifact/data_ingestion/raw_data', 'train_data_path': 'artifact/data_ingestion/train_data', 'test_data_path': 'artifact/data_ingestion/test_data'}}
artifact/
2025-04-28 18:51:58 - INFO: 1679058489 - Artifacts directory already exists at artifact/
2025-04-28 18:51:58 - INFO: utils - Directory artifact/data_ingestion created.
2025-04-28 18:51:58 - INFO: utils - Directory artifact/data_ingestion/raw_data created.
2025-04-28 18:51:58 - INFO: utils - Directory a

In [68]:
from urllib.request import urlretrieve



In [70]:
# download a csv file from a github repo
def download_csv_from_github(repo_url, file_path, save_path):
    """
    Downloads a CSV file from a GitHub repository.

    :param repo_url: URL of the GitHub repository (raw file URL)
    :param file_path: Path to the CSV file in the repository
    :param save_path: Local path to save the downloaded CSV
    """
    response = requests.get(repo_url)
    if response.status_code == 200:
        with open(save_path, 'wb') as file:
            file.write(response.content)
        print(f"CSV downloaded successfully to {save_path}")
    else:
        print(f"Failed to download CSV. Status code: {response.status_code}")

# Example usage
repo_url = "https://raw.githubusercontent.com/MainakRepositor/Datasets/refs/heads/master/Titanic.csv"
file_path = "data/example.csv"
save_path = "example.csv"
download_csv_from_github(repo_url, file_path, save_path)

CSV downloaded successfully to example.csv


In [4]:
import pandas as pd
example = {
        'Pclass':[1],
        'Sex': ['male'],
        'Age': [20],
        'SibSp':[0],
        'Parch':[0],
        'Fare':[7.25],
        'Embarked':['Q']
    }

pd.DataFrame(example).to_dict(index=False)

ValueError: 'index=False' is only valid when 'orient' is 'split' or 'tight'

In [3]:
import pandas as pd


df=pd.read_csv(r'C:\Users\ernes\Documents\ML Projects\ds-project-test\ds-project-test\artifact\data_ingestion\raw_data\Titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
for col in df.columns:
    if( df[col].dtype == 'object') and (col not in ['Name', 'Ticket']):
        print(col)
        print(df[col].unique())

Sex
['male' 'female']
Cabin
[nan 'C85' 'C123' 'E46' 'G6' 'C103' 'D56' 'A6' 'C23 C25 C27' 'B78' 'D33'
 'B30' 'C52' 'B28' 'C83' 'F33' 'F G73' 'E31' 'A5' 'D10 D12' 'D26' 'C110'
 'B58 B60' 'E101' 'F E69' 'D47' 'B86' 'F2' 'C2' 'E33' 'B19' 'A7' 'C49'
 'F4' 'A32' 'B4' 'B80' 'A31' 'D36' 'D15' 'C93' 'C78' 'D35' 'C87' 'B77'
 'E67' 'B94' 'C125' 'C99' 'C118' 'D7' 'A19' 'B49' 'D' 'C22 C26' 'C106'
 'C65' 'E36' 'C54' 'B57 B59 B63 B66' 'C7' 'E34' 'C32' 'B18' 'C124' 'C91'
 'E40' 'T' 'C128' 'D37' 'B35' 'E50' 'C82' 'B96 B98' 'E10' 'E44' 'A34'
 'C104' 'C111' 'C92' 'E38' 'D21' 'E12' 'E63' 'A14' 'B37' 'C30' 'D20' 'B79'
 'E25' 'D46' 'B73' 'C95' 'B38' 'B39' 'B22' 'C86' 'C70' 'A16' 'C101' 'C68'
 'A10' 'E68' 'B41' 'A20' 'D19' 'D50' 'D9' 'A23' 'B50' 'A26' 'D48' 'E58'
 'C126' 'B71' 'B51 B53 B55' 'D49' 'B5' 'B20' 'F G63' 'C62 C64' 'E24' 'C90'
 'C45' 'E8' 'B101' 'D45' 'C46' 'D30' 'E121' 'D11' 'E77' 'F38' 'B3' 'D6'
 'B82 B84' 'D17' 'A36' 'B102' 'B69' 'E49' 'C47' 'D28' 'E17' 'A24' 'C50'
 'B42' 'C148']
Embarked
['S' '

In [9]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [12]:
df['Fare'].max(), df['Fare'].min()

(np.float64(512.3292), np.float64(0.0))

In [13]:
df['Pclass'].unique()

array([3, 1, 2])

In [14]:
df['SibSp'].unique()

array([1, 0, 3, 4, 2, 5, 8])

In [15]:
df['Parch'].unique()

array([0, 1, 2, 5, 3, 4, 6])

In [16]:
%cd ..

c:\Users\ernes\Documents\ML Projects\ds-project-test\ds-project-test


In [17]:
import os
import time
import pandas as pd
from pathlib import Path
from serving import Serving
from src.ds_project.utils.utils import load_json, update_yaml, read_yaml
from src.ds_project.constants import PARAMS_FILE_PATH


def do_something_slow():
    time.sleep(5)


serving=Serving()
params_project=read_yaml(PARAMS_FILE_PATH)
params_models=pd.DataFrame(params_project['training']['MODELS'])
prediction=None
params_models['LOGISTIC_REGRESSION']['C'] = 1.0
print(params_project)
print(params_models)

2025-05-12 19:35:07 - INFO: __init__ - Logging setup complete.
2025-05-12 19:35:08 - INFO: utils - YAML file config\config.yaml loaded successfully.
2025-05-12 19:35:22 - INFO: utils - Binary file artifact\data_transformation\artifacts\preprocessing.pkl loaded successfully.
2025-05-12 19:35:22 - INFO: utils - Binary file artifact\model_validator\best_model\model.pkl loaded successfully.
2025-05-12 19:35:22 - INFO: utils - YAML file parameters.yaml loaded successfully.
{'transform_data': {'RANDOM_SEED': 86, 'TEST_SIZE': 0.2, 'DROPPED_COLUMNS': ['PassengerId', 'Name', 'Ticket', 'Cabin']}, 'training': {'TARGET_COLUMN': 'Survived', 'MODELS': {'LOGISTIC_REGRESSION': {'penalty': 'l2', 'C': 0.8, 'max_iter': 100, 'random_state': 86}, 'GRADIENT_BOOSTING': {'loss': 'log_loss', 'learning_rate': 0.1, 'n_estimators': 100, 'max_depth': 3, 'random_state': 86}, 'RANDOM_FOREST': {'n_estimators': 250, 'max_depth': 15, 'random_state': 86}}}, 'validator': {'threshold': 0.1}}
              LOGISTIC_REGRESS

C:\Users\ernes\AppData\Local\Temp\ipykernel_3852\3362545915.py:18: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  params_models['LOGISTIC_REGRESSION']['C'] = 1.0
C:\Users\ernes\AppData\Local\Temp\ipykernel_3852\3362545915.py:18: SettingWithCo

In [19]:
params_models

,LOGISTIC_REGRESSION,GRADIENT_BOOSTING,RANDOM_FOREST
penalty,l2,NaN,NaN
C,1.0,NaN,NaN
max_iter,100,NaN,NaN
random_state,86,86,86.0
loss,NaN,log_loss,NaN
learning_rate,NaN,0.1,NaN
n_estimators,NaN,100,250.0
max_depth,NaN,3,15.0


In [20]:
type(params_models)

pandas.core.frame.DataFrame

In [ ]:
from src.ds_project.constants import CONFIG_FILE_PATH


WindowsPath('parameters.yaml')

In [ ]:
'5'.isnumeric()

p = {
    'norm':'l2',
    'c':'5.0',
    's':'5'
}


def change_dict_dtypes(dict):
    new_dict={}
    for k,v in dict.items():
        if isinstance(v, str):
            if v.isnumeric():
                v = int(v)
            else:
                try:
                    v=float(v)
                except:
                    pass
        new_dict[k]=v

    return new_dict


In [33]:
change_dict_dtypes(p)

{'norm': 'l2', 'c': 5.0, 's': 5}

In [24]:
isinstance('s', str)

True

In [23]:
1.isnumeric()

<>:1: SyntaxWarning: invalid decimal literal
<>:1: SyntaxWarning: invalid decimal literal
C:\Users\ernes\AppData\Local\Temp\ipykernel_3852\3025934620.py:1: SyntaxWarning: invalid decimal literal
  1.isnumeric()


SyntaxError: invalid syntax (3025934620.py, line 1)